In [1]:
import pyspark
import math
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [2]:
sc = pyspark.SparkContext(appName="jojo")

In [3]:
sc

<SparkContext master=local[*] appName=jojo>

In [4]:
rdd = sc.sequenceFile("part-00000")

In [5]:
rdd.count()

36133

In [6]:
result = rdd.take(10)

In [7]:
def conta_palavras_doc(item):
    url, conteudo = item
    palavras = conteudo.strip().split()
    return [(palavra, 1) for palavra in set(palavras)]

def junta_contagens(nova_contagem, contagem_atual):
    return nova_contagem + contagem_atual

rdd_doc_freq = rdd.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)

In [8]:
def conta_uma_palavra(item, palavra):
    url, conteudo = item
    palavras = conteudo.strip().split()
    #count = 0
    lista = [palavra.lower() for palavra in palavras]
    if palavra in lista:
        return [item]
    return []


Rakin = rdd.filter(lambda x:"rakin" in x[1])
Yoda = rdd.filter(lambda x:"yoda" in x[1])
inter = Rakin.intersection(Yoda)

rdd_Rakin = Rakin.flatMap(conta_uma_palavra).reduceByKey(junta_contagens)
rdd_Yoda = Yoda.flatMap(conta_uma_palavra).reduceByKey(junta_contagens)
rdd_inter = inter.flatMap(conta_uma_palavra).reduceByKey(junta_contagens)

rdd_doc_rakin = Rakin.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)
rdd_doc_yoda = Yoda.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)
rdd_doc_inter = inter.flatMap(conta_palavras_doc).reduceByKey(junta_contagens)

In [9]:
rdd_doc_rakin.count()

11364

In [10]:
rdd_doc_yoda.count()

9151

In [11]:
rdd_doc_inter.count()

0

In [12]:
#rdd_doc_freq.count() #Numero de documentos

In [13]:
N = rdd.count()

In [14]:
DOC_COUNT_MIN = 5
DOC_COUNT_MAX = 0.7*N

def filtra_doc_freq(item):
    contagem = item[1]
    return (contagem < DOC_COUNT_MAX) and (contagem >= DOC_COUNT_MIN)

rdd_doc_rakin_freq_filtrado = rdd_doc_rakin.filter(filtra_doc_freq)
rdd_doc_yoda_freq_filtrado = rdd_doc_yoda.filter(filtra_doc_freq)
rdd_doc_inter_freq_filtrado = rdd_doc_inter.filter(filtra_doc_freq)

In [15]:
#rdd_doc_rakin_freq_filtrado.count() #Documentos filtrado no range definido

In [16]:
def computa_idf(item):
    palavra,contagem = item
    idf = math.log10(N/contagem)
    return (palavra,idf)

def computa_freq(item):
    palavra,contagem = item
    quant = math.log10(1+contagem)
    return (palavra,quant)

rakin_rdd_idf = rdd_doc_rakin_freq_filtrado.map(computa_idf)
yoda_rdd_idf = rdd_doc_yoda_freq_filtrado.map(computa_idf)
inter_rdd_idf = rdd_doc_inter_freq_filtrado.map(computa_idf)

rakin_rdd_freq = rdd_doc_rakin_freq_filtrado.map(computa_freq)
yoda_rdd_freq = rdd_doc_yoda_freq_filtrado.map(computa_freq)
inter_rdd_freq = rdd_doc_inter_freq_filtrado.map(computa_freq)

In [17]:
def relevancia(item):
    palavra, quant = item
    freq, idf = quant
    relevancia = freq*idf
    return (palavra, relevancia)

In [23]:
result = rakin_rdd_idf.collect()

In [24]:
#sorted(result, key=lambda x: x[1])[100:]

In [35]:
rdd_inter_join = inter_rdd_freq.join(inter_rdd_idf)
relevancia_inter = rdd_inter_join.map(relevancia)

inter_list = relevancia_inter.takeOrdered(100, key=lambda x: -x[1])

In [26]:
result1 = relevancia_inter.collect()

In [22]:
sorted(result, key=lambda x: x[1])[100:]

[]

In [27]:
stopwords = set(STOPWORDS)
stopwords.update(["da", "meu", "em", "você", "de", "ao", "os"])

In [32]:
def concat(item):
  summary = {}
  for s, c in item:
    summary[s] = c
    return summary

In [37]:
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800).generate(summary = concat(inter_list))

TypeError: expected string or bytes-like object